In [1]:
DATA_PATH = "../data/Processed/V3Explored.csv"
EXPORT_PATH = "../data/Processed/V3Processed.csv"
REPLACED_NAME = {
    'carpet_area': 'carpet_area_sq_ft',
    'estimated value': 'estimated value ($)',
    'sale price': 'sale price ($)'
}

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
df = pd.read_pickle(DATA_PATH)
df.shape

(10000, 12)

## Dealing with data  

In [4]:
df['date'] = pd.to_datetime(df['date'])

In [5]:
df.sample(3)

,date,year,locality,estimated value,sale price,property,residential,num_rooms,num_bathrooms,carpet_area,property_tax_rate,face
9790,2022-08-17,2022,NaN,375900.0,600000.0,?,Detached House,3,1,NaN,1.003979,West
2359,2012-07-26,2012,NaN,395710.0,580000.0,Single Family,Detached House,3,3,1031.0,1.021958,North
6996,2019-05-19,2019,Stamford,393090.0,713000.0,Single Family,Detached House,3,3,1056.0,1.025899,North


In [6]:
df['residential'].value_counts()

residential
Detached House    8296
Duplex            1014
Triplex            584
Fourplex           106
Name: count, dtype: int64

In [7]:
df.isna().sum()

date                    0
year                    0
locality             1255
estimated value      1229
sale price              0
property                0
residential             0
num_rooms               0
num_bathrooms           0
carpet_area          1282
property_tax_rate       0
face                    0
dtype: int64

In [8]:
def missing_values_table(df):
     for col in df.columns:
        print(col, df[col].nunique())

In [9]:
missing_values_table(df)

date 3089
year 14
locality 7
estimated value 7697
sale price 2619
property 5
residential 4
num_rooms 4
num_bathrooms 8
carpet_area 861
property_tax_rate 14
face 4


In [10]:
drop_cols = [ 'estimated value', 'carpet_area']
for col in drop_cols:
    df.drop(df[df[col].isnull()].index, inplace = True)

In [11]:
missing_values_table(df[drop_cols])

estimated value 6789
carpet_area 815


In [12]:
df.isna().sum()

date                   0
year                   0
locality             949
estimated value        0
sale price             0
property               0
residential            0
num_rooms              0
num_bathrooms          0
carpet_area            0
property_tax_rate      0
face                   0
dtype: int64

## Dealing with missing values

In [13]:
df['locality'] = df['locality'].fillna('Unknown')

In [14]:
df['property'] = df['property'].replace('?', np.nan) 

In [15]:
df['property'].value_counts()

property
Single Family    5205
Two Family        651
Three Family      370
Four Family        61
Name: count, dtype: int64

In [16]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Define the features to use for prediction with normalized column names
features = ['sale price', 'estimated value', 'carpet_area', 'num_bathrooms','num_rooms']

# Separate rows with and without missing 'property' values
df_missing = df[df['property'].isna()]
df_not_missing = df.dropna(subset=['property'])

# Prepare the data for modeling
X = df_not_missing[features]
y = df_not_missing['property']

# Split the data for training and testing (optional, for evaluation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict the missing 'property' values
X_missing = df_missing[features]
predicted_properties = model.predict(X_missing)

# Fill the missing values with predictions
df.loc[df['property'].isna(), 'property'] = predicted_properties

print(df['property'].value_counts())


property
Single Family    6326
Two Family        772
Three Family      453
Four Family        78
Name: count, dtype: int64


In [17]:
df.isnull().mean()*100

date                 0.0
year                 0.0
locality             0.0
estimated value      0.0
sale price           0.0
property             0.0
residential          0.0
num_rooms            0.0
num_bathrooms        0.0
carpet_area          0.0
property_tax_rate    0.0
face                 0.0
dtype: float64

In [18]:
df.duplicated().sum()

0

## DateTime Processing
-  Extract the monthe name from the data and add new column called monthe_name 

In [19]:
df['month_name'] = df['date'].dt.strftime('%B')

In [20]:
# Drop the 'date' column 
df.drop(columns=['date'], inplace=True)

In [21]:
df.sample(3)

,year,locality,estimated value,sale price,property,residential,num_rooms,num_bathrooms,carpet_area,property_tax_rate,face,month_name
3499,2014,Waterbury,156200.0,169900.0,Single Family,Detached House,3,1,1084.0,1.350819,South,July
9999,2022,Norwalk,237880.0,430000.0,Single Family,Detached House,3,1,927.0,1.003979,North,September
4486,2015,Unknown,339150.0,617000.0,Single Family,Detached House,3,3,934.0,1.340225,West,December


- Rename the column from 'carpet_area' to 'carpet_area_sq_ft'

In [22]:
df.rename(columns=REPLACED_NAME, inplace=True)

In [23]:
df.columns

Index(['year', 'locality', 'estimated value ($)', 'sale price ($)', 'property',
       'residential', 'num_rooms', 'num_bathrooms', 'carpet_area_sq_ft',
       'property_tax_rate', 'face', 'month_name'],
      dtype='object')

# Calculate tax value


In [24]:
df['tax_value'] = df['sale price ($)'] * df['property_tax_rate'] / 100


In [25]:
# Drop the 'property_tax_rate' column 
df.drop(columns=['property_tax_rate'], inplace=True)

In [26]:
df

,year,locality,estimated value ($),sale price ($),property,residential,num_rooms,num_bathrooms,carpet_area_sq_ft,face,month_name,tax_value
0,2009,Waterbury,111440.0,185000.0,Single Family,Detached House,3,3,996.0,South,January,1898.013050
1,2009,Unknown,73080.0,152000.0,Single Family,Detached House,3,1,935.0,North,January,1559.448560
2,2009,Waterbury,50540.0,105000.0,Single Family,Detached House,3,1,951.0,East,January,1077.250650
4,2009,Bridgeport,4775276.0,272900.0,Single Family,Detached House,3,1,971.0,East,January,2799.825737
7,2009,Norwalk,172620.0,409000.0,Single Family,Detached House,3,3,1004.0,South,January,4196.147770
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,2022,Unknown,216420.0,415000.0,Single Family,Detached House,3,3,1014.0,North,September,4166.512850
9995,2022,Unknown,155550.0,215000.0,Three Family,Triplex,6,5,2114.0,North,September,2158.554850
9997,2022,West Hartford,124740.0,225635.0,Single Family,Detached House,3,1,922.0,North,September,2265.328017
9998,2022,Unknown,1455230.0,2625000.0,Single Family,Detached House,3,2,977.0,North,September,26354.448750


# Export Data 

In [27]:
df.to_pickle(EXPORT_PATH)